In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import re
import random

In [4]:
df = pd.read_csv('language_sentences.csv', usecols=[0,1,2], delimiter=',')
df.head()

,deu,fra,eng
0,Lass uns etwas versuchen!,Lorsqu'il a demandé qui avait cassé la fenêtre...,Let's try something.
1,Ich muss schlafen gehen.,Je ne supporte pas ce type.,I have to go to sleep.
2,Was ist das?,Pour une fois dans ma vie je fais un bon geste...,Today is June 18th and it is Muiriel's birthday!
3,Heute ist der 18. Juni und das ist der Geburts...,Ne tenez aucun compte de ce qu'il dit.,Muiriel is 20 now.
4,"Herzlichen Glückwunsch zum Geburtstag, Muiriel!",Essayons quelque chose !,"The password is ""Muiriel""."


In [100]:
data = []

for _, column in enumerate(df.columns):
    for sentence in df[column]:
        
        sentence = re.sub('[!@#$?.;,''""]', '', sentence)
        sentence = re.sub('[0-9]', '', sentence)
        
        if column == "deu":
            data.append((sentence.lower().split(), "GERMAN"))
        elif column == "fra":
            data.append((sentence.lower().split(), "FRENCH"))
        elif column == "eng":
            data.append((sentence.lower().split(), "ENGLISH"))

In [120]:
random.shuffle(data)
data

[(['il', "s'est", 'révélé', 'être', 'un', 'espion'], 'FRENCH'),
 (['but', 'now', 'i', 'live', 'in', 'tokyo'], 'ENGLISH'),
 (['english', 'is', 'an', 'international', 'language'], 'ENGLISH'),
 (['i', 'am', 'good', 'at', 'speaking', 'english'], 'ENGLISH'),
 (['welche',
   'berühmten',
   'lieder',
   'hättet',
   'ihr',
   'gerne',
   'komponiert',
   'und',
   'warum'],
  'GERMAN'),
 (['il', 'progresse', 'bien', 'dans', 'ses', 'études'], 'FRENCH'),
 (['hier', 'ist', 'ein', 'brief', 'für', 'euch'], 'GERMAN'),
 (['toutes',
   'ces',
   'nouvelles',
   'caractéristiques',
   'de',
   'la',
   'france',
   'ressorties',
   'grâce',
   'à',
   'la',
   'révolution',
   'sont',
   'mises',
   'par',
   'écrit',
   'dans',
   'les',
   'deux',
   'documents',
   'fondamentaux',
   'pour',
   'le',
   'bon',
   'fonctionnement',
   'de',
   'l’état:',
   'la',
   'déclaration',
   'des',
   'droits',
   'de',
   'l’homme',
   'et',
   'du',
   'citoyen',
   'et',
   'la',
   'constitution'],
  '

In [124]:
n_total_samples = len(data)
n_training_samples = int(n_total_samples * 0.8)
n_test_samples = n_total_samples - n_training_samples

print(f'Number of total samples: {n_total_samples}\
      \nNumber of training samples: {n_training_samples}\
      \nNumber of test samples: {n_test_samples}')

Number of total samples: 221391      
Number of training samples: 177112      
Number of test samples: 44279


In [127]:
training_data = data[:n_training_samples]
print(len(training_data))

test_data = data[n_training_samples:]
print(len(test_data))

177112
44279


In [132]:
word_to_ix = {}
for sent, _ in training_data + test_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

# print(word_to_ix)

VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = 3

{'il': 0, "s'est": 1, 'révélé': 2, 'être': 3, 'un': 4, 'espion': 5, 'but': 6, 'now': 7, 'i': 8, 'live': 9, 'in': 10, 'tokyo': 11, 'english': 12, 'is': 13, 'an': 14, 'international': 15, 'language': 16, 'am': 17, 'good': 18, 'at': 19, 'speaking': 20, 'welche': 21, 'berühmten': 22, 'lieder': 23, 'hättet': 24, 'ihr': 25, 'gerne': 26, 'komponiert': 27, 'und': 28, 'warum': 29, 'progresse': 30, 'bien': 31, 'dans': 32, 'ses': 33, 'études': 34, 'hier': 35, 'ist': 36, 'ein': 37, 'brief': 38, 'für': 39, 'euch': 40, 'toutes': 41, 'ces': 42, 'nouvelles': 43, 'caractéristiques': 44, 'de': 45, 'la': 46, 'france': 47, 'ressorties': 48, 'grâce': 49, 'à': 50, 'révolution': 51, 'sont': 52, 'mises': 53, 'par': 54, 'écrit': 55, 'les': 56, 'deux': 57, 'documents': 58, 'fondamentaux': 59, 'pour': 60, 'le': 61, 'bon': 62, 'fonctionnement': 63, 'l’état:': 64, 'déclaration': 65, 'des': 66, 'droits': 67, 'l’homme': 68, 'et': 69, 'du': 70, 'citoyen': 71, 'constitution': 72, 'die': 73, 'vase': 74, 'fiel': 75, 'au

In [133]:
VOCAB_SIZE, NUM_LABELS

(75073, 3)

In [134]:
def make_bow_vector(sentence, word_to_ix):
    vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        vec[word_to_ix[word]] += 1
    return vec.view(1, -1)


def make_target(label, label_to_ix):
    return torch.LongTensor([label_to_ix[label]])

In [137]:
model = nn.Sequential(
    nn.Linear(VOCAB_SIZE, NUM_LABELS),
    nn.LogSoftmax(dim=1)
)

# for param in model.parameters():
#     print(param.shape)

# with torch.no_grad():
#     sample = data[0]
#     bow_vector = make_bow_vector(sample[0], word_to_ix)
#     log_preds = model(bow_vector)
#     print(log_preds)

label_to_ix = {"GERMAN": 0, "FRENCH":1, "ENGLISH": 2}

# Run on test data before we train, just to see a before-and-after
# with torch.no_grad():
#     for instance, label in test_data:
#         bow_vec = make_bow_vector(instance, word_to_ix)
#         log_probs = model(bow_vec)
#         print(log_probs)

# Print the matrix column corresponding to "ich"
# print(next(model.parameters())[:, word_to_ix["ich"]])

loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-1)

In [141]:
for epoch in range(1):

    for instance, label in training_data:
        model.zero_grad()
        
        bow_vec = make_bow_vector(instance, word_to_ix)
        target = make_target(label, label_to_ix)
        
        prediction = model(bow_vec)
        
        loss = loss_function(prediction, target)
        loss.backward()
        optimizer.step()
        
    if epoch % 10 == 0:
        print(f'epoch: {epoch}, loss: {loss}')

epoch: 0, loss: 0.00043561504571698606


In [142]:
with torch.no_grad():
    for instance, label in test_data:
        bow_vec = make_bow_vector(instance, word_to_ix)
        log_probs = model(bow_vec)
        print(log_probs)

tensor([[-4.3681e-04, -7.7917e+00, -1.0654e+01]])
tensor([[-3.9612, -0.0290, -4.6535]])
tensor([[-1.2717e+01, -9.7866e-05, -9.2634e+00]])
tensor([[-16.9740,   0.0000, -16.7309]])
tensor([[-2.8371e-05, -1.1003e+01, -1.1356e+01]])
tensor([[-19.4433, -21.4843,   0.0000]])
tensor([[-22.6019, -22.7121,   0.0000]])
tensor([[-30.4553,   0.0000, -25.8019]])
tensor([[-1.3616e+01, -9.8734e+00, -5.2689e-05]])
tensor([[-1.1607e+01, -5.8181e+00, -2.9867e-03]])
tensor([[-1.3113e-06, -1.4305e+01, -1.4087e+01]])
tensor([[-9.6273e+00, -9.7776e+00, -1.2266e-04]])
tensor([[-1.1516e+01, -1.8358e-05, -1.1690e+01]])
tensor([[-30.3614, -30.1158,   0.0000]])
tensor([[-24.2982, -24.2997,   0.0000]])
tensor([[  0.0000, -23.2734, -22.7155]])
tensor([[  0.0000, -22.9637, -21.7009]])
tensor([[-17.8655,   0.0000, -18.2583]])
tensor([[  0.0000, -18.9906, -21.2507]])
tensor([[-1.3446e-04, -1.2471e+01, -8.9431e+00]])
tensor([[-23.3338,   0.0000, -22.7725]])
tensor([[-24.4609, -22.6821,   0.0000]])
tensor([[-3.5130e-03

tensor([[-25.5985, -25.9781,   0.0000]])
tensor([[-3.4909, -2.5798, -0.1123]])
tensor([[-6.6438e+00, -6.5848e+00, -2.6868e-03]])
tensor([[-24.0964,   0.0000, -24.4008]])
tensor([[-1.1522e-03, -7.1740e+00, -7.8616e+00]])
tensor([[-18.9998,   0.0000, -27.7674]])
tensor([[-1.1438e+01, -1.5378e-05, -1.2283e+01]])
tensor([[-1.1408e-04, -9.5610e+00, -1.0040e+01]])
tensor([[-1.4867e+01, -7.1526e-07, -1.4925e+01]])
tensor([[-5.9605e-07, -1.7167e+01, -1.4270e+01]])
tensor([[-1.4388e+01, -1.1921e-06, -1.4335e+01]])
tensor([[  0.0000, -33.5059, -31.8927]])
tensor([[-8.0626e+00, -4.9460e-04, -8.6263e+00]])
tensor([[  0.0000, -17.0937, -16.9169]])
tensor([[-1.1082e+01, -1.1509e+01, -2.5391e-05]])
tensor([[-1.0710e+01, -1.0377e+01, -5.3524e-05]])
tensor([[-1.1778e+01, -9.6559e-06, -1.3105e+01]])
tensor([[-19.7278, -20.9882,   0.0000]])
tensor([[  0.0000, -18.2793, -18.1932]])
tensor([[-19.9506, -17.7270,   0.0000]])
tensor([[-1.5691e+01, -3.5763e-07, -1.5350e+01]])
tensor([[  0.0000, -23.5794, -24.1

tensor([[-19.9702,   0.0000, -21.5599]])
tensor([[-18.8123,   0.0000, -18.5002]])
tensor([[  0.0000, -20.2284, -19.8370]])
tensor([[-7.3449, -4.9143, -0.0080]])
tensor([[-20.8333,   0.0000, -20.0794]])
tensor([[-8.7894e+00, -2.6330e-04, -9.1068e+00]])
tensor([[-1.7013e+01, -4.7684e-07, -1.4541e+01]])
tensor([[-5.9605e-07, -1.4537e+01, -1.6210e+01]])
tensor([[-7.5363e+00, -1.0053e-03, -7.6598e+00]])
tensor([[-22.1623, -23.3019,   0.0000]])
tensor([[-1.0275e+01, -5.0782e-05, -1.1026e+01]])
tensor([[-2.3842e-07, -1.5264e+01, -1.7948e+01]])
tensor([[-9.5618e+00, -1.2945e-04, -9.7359e+00]])
tensor([[-8.5583e+00, -8.0916e+00, -4.9817e-04]])
tensor([[-1.4947e+01, -8.3446e-07, -1.4578e+01]])
tensor([[-1.8120e-05, -1.2129e+01, -1.1267e+01]])
tensor([[-21.0449,   0.0000, -21.4587]])
tensor([[-5.0068e-06, -1.2919e+01, -1.2894e+01]])
tensor([[-35.1661,   0.0000, -33.3041]])
tensor([[  0.0000, -20.2578, -19.1792]])
tensor([[-0.0120, -4.4535, -7.9963]])
tensor([[-5.3644e-06, -1.3044e+01, -1.2641e+01

tensor([[-1.4266e+01, -9.5367e-07, -1.4792e+01]])
tensor([[-2.2731e-04, -8.8274e+00, -9.4261e+00]])
tensor([[-7.5576e-05, -9.9983e+00, -1.0414e+01]])
tensor([[-6.5565e-06, -1.2505e+01, -1.2749e+01]])
tensor([[  0.0000, -16.8472, -19.5816]])
tensor([[-1.5516e+01, -3.5763e-07, -1.5665e+01]])
tensor([[  0.0000, -20.0605, -20.4938]])
tensor([[-22.9924,   0.0000, -22.7992]])
tensor([[-1.3632e+01, -1.4305e-06, -1.5503e+01]])
tensor([[-1.2433e+01, -1.4745e-04, -8.8493e+00]])
tensor([[-6.3958e+00, -1.0245e+01, -1.7055e-03]])
tensor([[-36.1084, -40.8009,   0.0000]])
tensor([[-9.7554e+00, -9.5604e+00, -1.2850e-04]])
tensor([[-2.0266e-06, -1.3721e+01, -1.3899e+01]])
tensor([[-9.4474e+00, -1.3994e-04, -9.7044e+00]])
tensor([[-33.3654,   0.0000, -32.9067]])
tensor([[-6.0649e+00, -8.2406e+00, -2.5900e-03]])
tensor([[-22.1477,   0.0000, -21.2266]])
tensor([[-1.0081e+01, -9.6172e+00, -1.0847e-04]])
tensor([[-3.4689e-05, -1.0663e+01, -1.1384e+01]])
tensor([[  0.0000, -18.6886, -18.1701]])
tensor([[-1.3

tensor([[  0.0000, -36.8256, -37.9262]])
tensor([[-0.6424, -1.0397, -2.1169]])
tensor([[-1.5824e+01, -2.3842e-07, -1.5865e+01]])
tensor([[-19.6726,   0.0000, -19.8904]])
tensor([[-1.4358e+01, -1.1921e-06, -1.4386e+01]])
tensor([[-20.3534,   0.0000, -20.2012]])
tensor([[-2.4830, -5.5816, -0.0913]])
tensor([[-9.5112e+00, -5.9929e+00, -2.5736e-03]])
tensor([[-2.0186e+01, -1.1921e-07, -1.6468e+01]])
tensor([[-19.2291,   0.0000, -18.7783]])
tensor([[-1.7491, -1.3771, -0.5555]])
tensor([[-1.5343e+01, -5.9605e-07, -1.4942e+01]])
tensor([[ -4.0661, -12.9021,  -0.0173]])
tensor([[-22.4203, -22.7169,   0.0000]])
tensor([[-1.1883e+01, -1.1669e+01, -1.5497e-05]])
tensor([[-1.0496e+01, -1.3938e+01, -2.8491e-05]])
tensor([[-9.6337e+00, -9.1334e+00, -1.7355e-04]])
tensor([[-8.9830e+00, -8.8319e+00, -2.7152e-04]])
tensor([[-25.1976, -22.7289,   0.0000]])
tensor([[-9.9654e-05, -9.7208e+00, -1.0137e+01]])
tensor([[-1.2045e+01, -7.2717e-06, -1.3430e+01]])
tensor([[-19.1015,   0.0000, -19.1612]])
tensor([

...

tensor([[-3.8147e-06, -1.5114e+01, -1.2537e+01]])
tensor([[  0.0000, -22.3011, -21.1326]])
tensor([[-6.1022e+00, -9.0255e+00, -2.3611e-03]])
tensor([[-1.4695e+01, -8.3446e-07, -1.4677e+01]])
tensor([[-6.6198e+00, -9.7445e+00, -1.3933e-03]])
tensor([[-1.3282e+01, -2.8610e-06, -1.3660e+01]])
tensor([[-4.5656e-05, -1.0695e+01, -1.0681e+01]])
tensor([[  0.0000, -20.5797, -20.5282]])
tensor([[-7.7108e+00, -7.4383e-04, -8.1266e+00]])
tensor([[-1.4693e+01, -1.5028e+01, -7.1526e-07]])
tensor([[-1.6253e+01, -1.9207e+01, -1.1921e-07]])
tensor([[-23.0734, -23.2169,   0.0000]])
tensor([[-1.7643e-05, -1.1675e+01, -1.1596e+01]])
tensor([[-1.2471e+01, -5.8412e-06, -1.3107e+01]])
tensor([[-1.7685e-03, -6.7191e+00, -7.4887e+00]])
tensor([[-18.2775, -19.4703,   0.0000]])
tensor([[-1.7302e+01, -5.9605e-07, -1.4312e+01]])
tensor([[-2.2888e-05, -1.0891e+01, -1.2360e+01]])
tensor([[-18.4450, -16.9221,   0.0000]])
tensor([[-1.0787e+01, -3.3140e-05, -1.1285e+01]])
tensor([[-1.3302e+01, -1.3128e+01, -3.6955e-0

tensor([[-2.3842e-06, -1.4355e+01, -1.3255e+01]])
tensor([[-9.1479e+00, -2.0454e-04, -9.2301e+00]])
tensor([[-17.4277, -18.1179,   0.0000]])
tensor([[-1.4485e+01, -1.3208e+01, -2.3842e-06]])
tensor([[-18.9795,   0.0000, -18.6768]])
tensor([[-30.5456,   0.0000, -29.6786]])
tensor([[-9.0999e-04, -7.3583e+00, -8.2088e+00]])
tensor([[-32.2205,   0.0000, -36.9376]])
tensor([[-1.4124e+01, -1.4305e-06, -1.4121e+01]])
tensor([[-2.4197e-04, -8.7892e+00, -9.3193e+00]])
tensor([[-2.8610e-06, -1.5385e+01, -1.2853e+01]])
tensor([[-38.6933,   0.0000, -38.3182]])
tensor([[-17.5513,   0.0000, -17.9749]])
tensor([[-21.3669, -24.0447,   0.0000]])
tensor([[-1.4670e+01, -1.2636e-05, -1.1316e+01]])
tensor([[-21.9569,   0.0000, -22.1622]])
tensor([[-1.2914e+01, -1.0810e+01, -2.2650e-05]])
tensor([[-22.9588,   0.0000, -23.1666]])
tensor([[-1.1921e-07, -1.8933e+01, -1.6016e+01]])
tensor([[-1.9076e+01, -1.6260e+01, -1.1921e-07]])
tensor([[-2.2064e+01, -5.9605e-07, -1.4345e+01]])
tensor([[-4.8898, -0.0133, -5.1

tensor([[-5.6028e-06, -1.2696e+01, -1.2898e+01]])
tensor([[-0.0925, -2.9525, -3.3214]])
tensor([[-1.4738e+01, -1.2353e+01, -4.7684e-06]])
tensor([[-3.0636e-05, -1.0721e+01, -1.1661e+01]])
tensor([[-18.0769, -18.9244,   0.0000]])
tensor([[-1.3113e-06, -1.4083e+01, -1.4242e+01]])
tensor([[-20.6617,   0.0000, -21.0219]])
tensor([[-24.8627, -25.6457,   0.0000]])
tensor([[-1.1921e-06, -1.4166e+01, -1.4509e+01]])
tensor([[-1.4626e+01, -7.1526e-07, -1.5467e+01]])
tensor([[-18.2310,   0.0000, -18.3871]])
tensor([[-41.5374,   0.0000, -42.7609]])
tensor([[-7.5102e-06, -1.2305e+01, -1.2736e+01]])
tensor([[-5.3644e-06, -1.3649e+01, -1.2390e+01]])
tensor([[-1.3444e+01, -2.2650e-06, -1.4002e+01]])
tensor([[-23.7748,   0.0000, -19.8872]])
tensor([[  0.0000, -19.2053, -20.8813]])
tensor([[-1.3432e+01, -1.9073e-06, -1.4607e+01]])
tensor([[-2.7360, -2.3557, -0.1739]])
tensor([[-20.4994, -21.2052,   0.0000]])
tensor([[-9.5367e-07, -1.4844e+01, -1.4398e+01]])
tensor([[-7.4537e+00, -7.2503e+00, -1.2901e-03

tensor([[-22.9927, -24.0978,   0.0000]])
tensor([[-1.5552e+01, -1.1921e-07, -1.6701e+01]])
tensor([[-1.1921e-07, -1.5620e+01, -1.6792e+01]])
tensor([[-20.2747, -19.6768,   0.0000]])
tensor([[-18.3037, -18.9061,   0.0000]])
tensor([[-7.8439e+00, -5.6465e-04, -8.6659e+00]])
tensor([[-1.2686e+01, -1.2606e+01, -6.4373e-06]])
tensor([[-5.9605e-07, -1.5422e+01, -1.4931e+01]])
tensor([[-2.0265e-05, -1.1565e+01, -1.1437e+01]])
tensor([[  0.0000, -31.7604, -27.4172]])
tensor([[-20.7307,   0.0000, -20.0797]])
tensor([[  0.0000, -22.0501, -21.2620]])
tensor([[-8.1904e+00, -5.6743e+00, -3.7174e-03]])
tensor([[-1.5300e+01, -1.9150e+01, -2.3842e-07]])
tensor([[-4.7684e-07, -1.5137e+01, -1.5427e+01]])
tensor([[-0.0208, -4.2413, -5.0801]])
tensor([[-1.4756e+01, -4.7684e-07, -1.5918e+01]])
tensor([[-28.0702, -28.6073,   0.0000]])
tensor([[-9.7618e+00, -1.0583e+01, -8.2966e-05]])
tensor([[-4.5299e-05, -1.0397e+01, -1.1124e+01]])
tensor([[-1.7155e+01, -1.6797e+01, -1.1921e-07]])
tensor([[-1.7522e-04, -8.

tensor([[-7.6555e+00, -1.0378e-03, -7.4807e+00]])
tensor([[-2.5198e-04, -8.8309e+00, -9.1534e+00]])
tensor([[  0.0000, -25.8098, -25.1938]])
tensor([[-29.4760, -29.3401,   0.0000]])
tensor([[-6.7113e-05, -1.0330e+01, -1.0276e+01]])
tensor([[-1.1921e-06, -1.4179e+01, -1.4521e+01]])
tensor([[-9.6559e-06, -1.2170e+01, -1.2329e+01]])
tensor([[-9.3599e+00, -9.0613e+00, -2.0216e-04]])
tensor([[-1.2145e+01, -1.1802e-05, -1.1959e+01]])
tensor([[-7.3501e-04, -7.6992e+00, -8.1750e+00]])
tensor([[-7.8623e+00, -4.9293e-04, -9.1353e+00]])
tensor([[-0.0188, -4.5411, -4.8313]])
tensor([[-23.1710,   0.0000, -21.2361]])
tensor([[-5.6930e-04, -7.8682e+00, -8.5872e+00]])
tensor([[-20.2113, -19.8220,   0.0000]])
tensor([[  0.0000, -22.7713, -22.2214]])
tensor([[-21.9855, -26.8386,   0.0000]])
tensor([[-1.2856e+01, -1.2829e+01, -5.2452e-06]])
tensor([[-3.8925, -0.0209, -8.2180]])
tensor([[-21.9834,   0.0000, -25.8262]])
tensor([[-6.2985e+00, -3.2837e-03, -6.5436e+00]])
tensor([[-1.0594e+01, -1.0814e+01, -4

In [153]:
language = ["GERMAN", "FRENCH", "ENGLISH"]
probs = next(model.parameters())[:, word_to_ix["ne"]].tolist()
max_index = probs.index(max(probs))
print(language[max_index])

FRENCH
